### Get Stock Data and Returns

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np


# Get the tickers for SP-100
url = "https://en.wikipedia.org/wiki/S%26P_100"

# Read the tables on the page
tables = pd.read_html(url)

tickers = tables[2]['Symbol'].tolist()
tickers = [ticker.replace('.', '-') for ticker in tickers]


# Download weekly adjusted close prices
stock_prices         = yf.download(tickers, start="2020-01-01", end="2023-12-31", auto_adjust = False)
stock_prices = stock_prices.resample('W').last()
stock_prices.index   = stock_prices.index.tz_localize(None)      # change yf date format to match pdr
stock_prices         = stock_prices.filter(like='Adj Close')

# Drop columns with too many missing values (e.g. due to IPOs)
stock_prices = stock_prices.dropna(axis=1, thresh=int(0.9 * len(stock_prices)))

returns = stock_prices.pct_change().dropna().rename(columns={"Adj Close": "Return"})

[*********************100%***********************]  101 of 101 completed


In [2]:

returns = returns.xs('Return', axis=1, level=0)
returns.columns.name = None
print(returns)

                AAPL      ABBV       ABT       ACN      ADBE       AIG  \
Date                                                                     
2020-01-12  0.043372  0.004059 -0.006171 -0.010200  0.024110  0.013045   
2020-01-19  0.027068  0.001447  0.047071  0.011311  0.029222  0.016721   
2020-01-26 -0.001318 -0.050682  0.015731 -0.003011  0.004661 -0.034782   
2020-02-02 -0.027646 -0.030165 -0.036062 -0.016110 -0.000655 -0.015668   
2020-02-09  0.036443  0.139102  0.008377  0.031041  0.042576  0.063669   
...              ...       ...       ...       ...       ...       ...   
2023-12-03  0.006685  0.034182  0.019539  0.012035 -0.011236  0.011655   
2023-12-10  0.023374  0.040932 -0.003528 -0.002455 -0.004016 -0.006670   
2023-12-17  0.009504  0.031886  0.026600  0.020520 -0.041524  0.012771   
2023-12-24 -0.020094  0.005843  0.016218  0.029929  0.024064  0.013937   
2023-12-31 -0.005527  0.000194  0.009539 -0.009987 -0.003591  0.012252   

                 AMD      AMGN       

In [3]:
# Define how much of the data is out-of-sample
split_point = int(0.7 * len(returns))
out_sample = returns.iloc[split_point:].copy()

# Number of periods you want
m = 4
n_rows = len(out_sample)
period_length = n_rows // m

# Create period labels
periods = np.repeat(np.arange(1, m + 1), period_length)
remainder = n_rows - len(periods)
if remainder > 0:
    periods = np.append(periods, [m] * remainder)

# Add period column
out_sample['period'] = periods


is_windows = []
period_lengths = out_sample['period'].value_counts().sort_index()

for i in range(1, m + 1):    
    # Get the first date of this OOS window
    start_date = out_sample[out_sample['period'] == i].index[0]

    # Define IS window: go back `split` number of rows ending at start_date (non-inclusive)
    is_window = returns.loc[:start_date].iloc[-split_point:]
    is_windows.append(is_window)

In [4]:
for i in range(m):
    window_returns = is_windows[i]

    flat_returns = window_returns.copy()
    flat_returns.columns = window_returns.columns

    # Calculate correlations if not done yet
    correlations = flat_returns.corr()
    tickers = [col[1] for col in stock_prices.columns]
    x_matrix = pd.DataFrame(np.zeros((len(tickers), len(tickers))), index=tickers, columns=tickers)
    q = 10

    # Write to AMPL-style .dat file
    file_name = "max_corr_" + str(i) + ".txt"
    with open(file_name, "w") as f:
        f.write("set STOCKS := " + " ".join(tickers) + " ;\n\n")

        f.write("param q := " + str(q) + " ;\n\n")

        f.write("param r:\n    " + " ".join(tickers) + " :=\n")
        for t1 in tickers:
            row = " ".join(f"{correlations.loc[t1, t2]:.4f}" for t2 in tickers)
            f.write(f"{t1} {row}\n")
        f.write(";\n") 


# Run AMPL Code At This Point

In [5]:

all_results = []
current_result = []

with open("max_correlation_results.txt", 'r') as file:
    for line in file:
        line = line.strip()
        if line.lower().startswith("result"):
            if current_result:
                all_results.append(current_result)
                current_result = []
        else:
            parts = line.split()
            if len(parts) == 2 and parts[1] == '1':
                current_result.append(parts[0])

# Append the last result if not already appended
if current_result:
    all_results.append(current_result)

print(all_results)

periods = []
tickers = []
weights_list = []

for i in range(m):
    market_caps = {}
    selected_tickers = all_results[i]
    for ticker in selected_tickers:
        try:
            info = yf.Ticker(ticker).info
            market_caps[ticker] = info.get('marketCap', 0)
        except Exception as e:
            print(f"Error retrieving {ticker}: {e}")
            market_caps[ticker] = 0
            
    total_market_value = sum(market_caps.values())

    # Compute weights
    for ticker in selected_tickers:
        cap = market_caps[ticker]
        weight = cap / total_market_value if total_market_value > 0 else 0
        periods.append(i + 1)
        tickers.append(ticker)
        weights_list.append(weight)

weights_df = pd.DataFrame({
    'period': periods,
    'Ticker': tickers,
    'Weight': weights_list
})  
print(weights_df)



[['BLK', 'HON', 'JNJ', 'MDLZ', 'MET', 'MSFT', 'NVDA', 'SO', 'WMT', 'XOM'], ['ACN', 'CVX', 'HON', 'JNJ', 'JPM', 'MDLZ', 'MSFT', 'SO', 'TMO', 'V'], ['BAC', 'BLK', 'CVX', 'GD', 'JNJ', 'KO', 'MSFT', 'NVDA', 'TMO', 'V'], ['BAC', 'BLK', 'CVX', 'GD', 'JNJ', 'KO', 'MSFT', 'TMO', 'V', 'WMT']]
    period Ticker    Weight
0        1    BLK  0.017622
1        1    HON  0.016850
2        1    JNJ  0.045952
3        1   MDLZ  0.010732
4        1    MET  0.006390
5        1   MSFT  0.395775
6        1   NVDA  0.341773
7        1     SO  0.012254
8        1    WMT  0.096653
9        1    XOM  0.056000
10       2    ACN  0.032411
11       2    CVX  0.041021
12       2    HON  0.023356
13       2    JNJ  0.063695
14       2    JPM  0.118993
15       2   MDLZ  0.014875
16       2   MSFT  0.548584
17       2     SO  0.016986
18       2    TMO  0.027111
19       2      V  0.112968
20       3    BAC  0.037234
21       3    BLK  0.017339
22       3    CVX  0.029120
23       3     GD  0.008821
24       3    J

In [6]:
stock_set = set([item for sublist in all_results for item in sublist])
stock_set.add('period')
# Filter out_sample to only have the tickers present in stock_set
#out_sample_filtered = out_sample[out_sample.columns.intersection(stock_set)]

out_sample_tall = out_sample.reset_index().melt(id_vars=["Date", "period"], var_name="Ticker", value_name="Return")
print(out_sample_tall)

           Date  period Ticker    Return
0    2022-10-23       1   AAPL  0.064244
1    2022-10-30       1   AAPL  0.057513
2    2022-11-06       1   AAPL -0.109994
3    2022-11-13       1   AAPL  0.081804
4    2022-11-20       1   AAPL  0.010621
...         ...     ...    ...       ...
6295 2023-12-03       4    XOM -0.015109
6296 2023-12-10       4    XOM -0.033401
6297 2023-12-17       4    XOM  0.013561
6298 2023-12-24       4    XOM  0.010010
6299 2023-12-31       4    XOM -0.018938

[6300 rows x 4 columns]


In [7]:
portfolio = pd.merge(out_sample_tall, weights_df, on=['period', 'Ticker'], how='inner')
portfolio['Weighted_Return'] = portfolio['Return'] * portfolio['Weight']
portfolio_return = portfolio.groupby('Date')['Weighted_Return'].sum().reset_index()
portfolio_return = portfolio_return.rename(columns={'Weighted_Return': 'Portfolio_Return'})


# Analysis

In [8]:
market_caps = {}

selected_tickers = returns.columns
for ticker in selected_tickers:
    try:
        info = yf.Ticker(ticker).info
        market_caps[ticker] = info.get('marketCap', 0)
    except Exception as e:
        print(f"Error retrieving {ticker}: {e}")
        market_caps[ticker] = 0
        
total_market_value = sum(market_caps.values())


In [9]:
tickers = []
weights_list = []
# Compute weights
for ticker in selected_tickers:
    cap = market_caps[ticker]
    weight = cap / total_market_value
    tickers.append(ticker)
    weights_list.append(weight)

sp_weights = pd.DataFrame({
    'Ticker': tickers,
    'Weight': weights_list
})  


In [10]:
sp100 = pd.merge(out_sample_tall, sp_weights, on=['Ticker'], how='inner')
sp100['Weighted_Return'] = sp100['Return'] * sp100['Weight']
sp100_returns = sp100.groupby('Date')['Weighted_Return'].sum().reset_index()
sp100_returns = sp100_returns.rename(columns={'Weighted_Return': 'Portfolio_Return'})


In [11]:
print(sp100_returns['Portfolio_Return'].corr(portfolio_return['Portfolio_Return']))

0.8502563099533982


In [12]:
qs = [5, 15, 20]
all_portfolio_returns = {}
all_weights = {}

for q in qs:
    all_results = []

    for i in range(m):
        window_returns = is_windows[i]
        correlations = window_returns.corr()
        tickers = list(window_returns.columns)

        file_name = f"max_corr_q{q}_period{i}.dat"
        with open(file_name, "w") as f:
            f.write(f"set STOCKS := {' '.join(tickers)} ;\n\n")
            f.write(f"param q := {q} ;\n\n")
            f.write("param r:\n    " + " ".join(tickers) + " :=\n")
            for t1 in tickers:
                row = " ".join(f"{correlations.loc[t1, t2]:.4f}" for t2 in tickers)
                f.write(f"{t1} {row}\n")
            f.write(";\n")

RUN AMPL